In [1]:
#!pip install langchain
#!pip install pinecone-client
#!pip install openai
#!pip install matplotlib
#%pip install --upgrade tiktoken
#%pip install --upgrade openai

In [2]:
# # Define your environment variables as a dictionary
# env_variables = {
#     'OPENAI_API_KEY': OPENAI_API_KEY,
#     'PINECONE_API_KEY': PINECONE_API_KEY,
#     'PINECONE_ENV': PINECONE_ENV,
#     # Add more variables as needed
# }

# # Write the environment variables to .env file
# with open('.env', 'w') as f:
#     for key, value in env_variables.items():
#         f.write(f'{key}={value}\n')

# Load dependencies

In [3]:
import pandas as pd
import os
import re
import matplotlib.pyplot as plt
from datetime import datetime
import statistics
import time

from langchain.vectorstores import Pinecone
from langchain.embeddings.openai import OpenAIEmbeddings
from langchain.chat_models import ChatOpenAI
#from langchain.llms import OpenAI
from langchain.memory import VectorStoreRetrieverMemory
#from langchain.chains import ConversationChain
from langchain.prompts import PromptTemplate
from langchain.memory import ConversationBufferMemory
from langchain.chains import ConversationalRetrievalChain
from langchain.chains import RetrievalQAWithSourcesChain

import pinecone

/home/ubuntu/.local/lib/python3.10/site-packages/pinecone/index.py:4: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  from tqdm.autonotebook import tqdm


In [4]:
OPENAI_API_KEY= 
PINECONE_API_KEY=
PINECONE_ENV= 

In [5]:
#config
from pydantic_settings import BaseSettings
from dotenv import load_dotenv
import os

load_dotenv()  # Take environment variables from .env.


class Settings(BaseSettings):
    OPENAI_API_KEY: str = os.getenv("OPENAI_API_KEY", "default_openai_api_key")
    PINECONE_API_KEY: str = os.getenv("PINECONE_API_KEY", "default_pinecone_api_key")
    PINECONE_ENV: str = os.getenv("PINECONE_ENV", "gcp-starter")
    INDEX_NAME: str = os.getenv("INDEX_NAME", "brainsoft")
    EMBEDDING_NAME: str = os.getenv("EMBEDDING_NAME", "default_ambeddings")
    LLM_NAME: str = os.getenv("LLM_NAME", "default_LLM")
    MONGO_DB_KEY: str = os.getenv("MONGO_DB_KEY", "default_MONGO_DB_KEY")


# Instantiate settings to be imported by other modules
settings = Settings()

In [7]:
import logging


# Handling class
class UpdateError(Exception):
    """
    A custom exception class for handling update-related errors.

    This class extends the base Exception class and is used to raise exceptions specifically related to update processes in the application, such as dat

    Args:
        message (str): The error message describing what went wrong during the update process.
        status_code (int): The HTTP status code associated with this error, indicating the nature of the error.

    Upon initialization, this class logs the error message using the standard logging module.

    Attributes:fv
        message (str): Stores the error message.
        status_code (int): Stores the HTTP status code associated with the error.
    """

    def __init__(self, message, status_code):
        super().__init__(message)  # Initialize the base Exception class first
        self.message = message
        self.status_code = status_code
        self.log_error()  # Log the error after initializing

    def log_error(self):
        logging.error(self.message)


In [19]:
#utils
#from config import settings
import pinecone
import openai
# from utils.callback_handler_agent import *
# from utils.callback_handler_chain import *
# from utils.error_handler import *
from langchain.embeddings.openai import OpenAIEmbeddings
from langchain.chat_models import ChatOpenAI
from langchain.vectorstores import Pinecone
from langchain.tools import DuckDuckGoSearchRun, Tool
from langchain.agents import initialize_agent, AgentType
from langchain.callbacks.streaming_stdout import StreamingStdOutCallbackHandler
from langchain.agents.agent_toolkits import create_retriever_tool
from langchain.prompts import PromptTemplate
from langchain.chains import LLMChain


def setup_conversational_chain(settings: object):
    """
    Initializes the conversational chain with various tools and configurations.

    Args:
        settings (object): Application settings containing configuration details.

    Returns:
        tuple: A tuple containing the agent, retriever, and language model chain.

    This function sets up the conversational agent with various tools (like retrievers and search functions)
    and configures the language model chain. It handles the initialization of the database, vector database,
    language model, and other components required for the conversational chain.

    Raises:
        UpdateError: If there is an error during the initialization of any component.
    """
    global agent
    global retriever
    global llm

    tools = []

    # Initialize database

    try:
        pinecone.init(
            api_key=settings.PINECONE_API_KEY, environment=settings.PINECONE_ENV
        )

        embeddings_model = OpenAIEmbeddings(
            model=settings.EMBEDDING_NAME, openai_api_key=settings.OPENAI_API_KEY
        )

        vectordb = Pinecone.from_existing_index(settings.INDEX_NAME, embeddings_model)

    except Exception as e:
        raise UpdateError(f"Error during initialization of vector database: {e}", 401)

    # Initialize database LLM model

    try:
        llm = ChatOpenAI(
            openai_api_key=settings.OPENAI_API_KEY,
            model_name=settings.LLM_NAME,
            temperature=0,
            streaming=True,
            callbacks=[StreamingStdOutCallbackHandler()],
        )
    except Exception as e:
        raise UpdateError(f"Error during initialization of LLM: {e}", 402)
    # Prepare retriever

    try:
        retriever = vectordb.as_retriever(
            search_type="similarity_score_threshold",
            search_kwargs={"score_threshold": 0.1}, # , "k": 1
        )
    except Exception as e:
        raise UpdateError(f"Error during initialization of retriever: {e}", 403)

    # Initialize tools

    try:
        tool_retrieve = create_retriever_tool(
            retriever,
            "product_search",
            "Searches and returns products regarding the pinterest fashion that meets all requirements (if provided) such as age, gender, location, brand, price, availability. Focus on high rating of products first and click_rate second!",
        )

        search = DuckDuckGoSearchRun()
        search_tool = Tool(
            name="DuckDuckGo",
            func=search,  # .run
            description="This tool is used when you need to do a search on the internet to find information that another tool product_search can't find.",
        )

        tools.append(tool_retrieve)
        tools.append(search_tool)

        #        Initialize tools
        agent = initialize_agent(
            agent=AgentType.CHAT_CONVERSATIONAL_REACT_DESCRIPTION, <-
            tools=tools,
            llm=llm,
            verbose=True,
            max_iterations=10,
            early_stopping_method="generate",
            # memory=memory,
            return_intermediate_steps=False,
        )

    except Exception as e:
        raise UpdateError(f"Error during initialization of toolls an agent: {e}", 404)
    # Set template for history input

    try:
        new_string = "\n\n\nCHAT HISTORY AND USER'S INPUT\n-----------------------------\nHere is the Chat history followed by user's input"
        old_string = (
            "\n```\n\nUSER'S INPUT\n--------------------\nHere is the user's input"
        )
        template_new = agent.agent.llm_chain.prompt.messages[2].prompt.template.replace(
            old_string, new_string
        )

        agent.agent.llm_chain.prompt.messages[2].prompt.template = template_new

    except Exception as e:
        raise UpdateError(f"Unable to set custom template: {e}", 405)

    try:
        template = (
            """The system should generate a natural language response recommending products with justifications"""
            """(e.g., ”Based on your [requirements], we recommend product X from brand Y because it"""
            """has a high rating and is within your budget”), ensuring it is fully"""
            """supported by the provided context and make sure each product from context must be"""
            """mentioned! If any product doesnt meet any requirement, mention it. For example, selected product"""
            """ is a bit more expensive but it fits your other requirements perfectly!
        Context: {context}
        Question:{input}
        Answer: """
        )

        prompt = PromptTemplate(template=template, input_variables=["input", "context"])

        llm_chain = LLMChain(prompt=prompt, llm=llm)

    except Exception as e:
        raise UpdateError(f"Unable to set OpenAI chain: {e}", 406)
    return agent, retriever, llm_chain


In [20]:
agent, retriever, llm = setup_conversational_chain(settings)

In [34]:
query="Recomment shoes for male 800 in Slovakia  but my butget is not more than 80 dolars!"
#query="What is IBM ?"

In [ ]:
agent.invoke({"input": query,"chat_history":[]})